In [2]:
import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers

In [3]:
# Prepare data
train = dc.data.DiskDataset('tf_datasets/train')
valid = dc.data.DiskDataset('tf_datsets/valid')

In [4]:
# Build the architecture

features = tf.keras.Input(shape=(101, 4))

prev = features
for i in range(3):
    convLayer = layers.Conv1D(
        filters=15,
        kernel_size=10,
        activation=tf.nn.relu,
        padding='same')
    prev = convLayer(prev)
    dropoutLayer = layers.Dropout(rate=0.5)
    prev = dropoutLayer(prev)

logitsLayer = layers.Dense(units=1)
logits = logitsLayer(layers.Flatten()(prev))
# sets to 0/1 as opposed to a range of probability
output = layers.Activation(tf.math.sigmoid)(logits)

keras_model = tf.keras.Model(inputs=features, outputs=[output, logits])
model = dc.models.KerasModel(
    keras_model,
    loss=dc.models.losses.SigmoidCrossEntropy(),
    output_types=['prediction', 'loss'],
    batch_size=1000,
    model_dir='tf_model'  # TODO: what's this? Is this to store the generated model?
)

In [ ]:
# Train our model
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
for i in range(20):
    model.fit(train, nb_epoch=10)
    print(model.evaluate(train, [metric]))
    print(model.evaluate(valid, [metric]))